In [170]:
import numpy
import os
import random

In [184]:
os.chdir('/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/AI Project/assignment/example-problems')

In [172]:
def find_start_position(map):
    start_x, start_y = None, None
    #max_height, max_width = len(map), len(map[0])
    for y, row in enumerate(map):
        for x, char in enumerate(row):
            if char == 'S':
                start_x, start_y = x, y
                break
        if start_x is not None:  # Break outer loop if start found
            break

    #if start_x is None:
        #raise ValueError("Start position 'S' not found in the map.")
    
    return start_x,start_y

In [173]:
def read_map_from_file(filename):
    #initial_position_known=True
    with open(filename, 'r') as file:
        content = file.readlines()

    task = content[0].strip()
    
    if task == "FIND PLAN":
        # Remove only the newline character at the end if it exists
        map_data = [line[:-1] if line.endswith('\n') else line for line in content[1:]]
        plan = ""
        try: 
            start_x,start_y=find_start_position(map_data)
        except Exception as e:
            print(f"Could not find the starting position: {e}")
            start_x,start_y=None, None

    elif task == "CHECK PLAN":
        plan = content[1].strip()
        map_data = [line[:-1] if line.endswith('\n') else line for line in content[2:]]
        try: 
            start_x,start_y=find_start_position(map_data)
        except Exception as e:
            print(f"Could not find the starting position: {e}")
            start_x,start_y=None,None
    else:
        raise ValueError("Unknown task specified in the file")
    
    return task, plan, map_data,start_x,start_y

In [174]:
def check_all_clean(map,cleaned_squares):
    missed_squares = []
    for y, row in enumerate(map):
        for x, char in enumerate(row):
            if char != 'X' and (x, y) not in cleaned_squares:
                missed_squares.append((x, y))
    return missed_squares

In [175]:
def execute_plan(plan, map,start_x,start_y):
    height, width = len(map), len(map[0])
    #print("height and with:",height,"-",width)
    if plan == "":
        return None
    else:
        # Find starting position and calculate maximum width
        #start_x,start_y=find_start_position(map)
        cleaned_squares = set([(start_x, start_y)])  # Squares cleaned by the vacuum
        
        x, y = start_x, start_y  # Initialize vacuum's position to the starting position
        #print("starting position",x,y)
        # Assuming map_width and map_height are the dimensions of the map
        for instruction in plan:
            #print("instruction",instruction)
            new_x, new_y = x, y
            if instruction == 'N':
                new_y = (y - 1) % height
            elif instruction == 'E':
                new_x = (x + 1) % width
            elif instruction == 'S':
                new_y = (y + 1) % height
            elif instruction == 'W':
                new_x = (x - 1) % width
            #print((new_x,new_y))
            # Check for walls
            if map[new_y][new_x] != 'X':
                x, y = new_x, new_y
            cleaned_squares.add((x, y))
            #print("cleaned",(x,y))
        # Check if all non-wall squares are cleaned
        missed_squares=check_all_clean(map,cleaned_squares)
        
        return missed_squares



In [176]:
def find_potential_starts(map_data):
    potential_starts = []
    for y, row in enumerate(map_data):
        for x, char in enumerate(row):
            if char != 'X':  # Assuming any non-wall character is a potential starting point
                potential_starts.append((x, y))
    return potential_starts


In [177]:
def execute_plan_for_unknown_start(plan,map):
    potential_starts = find_potential_starts(map)
    all_missed_squares = set()
    
    for start_point in potential_starts:
        start_x,start_y=start_point
        missed_squares = execute_plan(plan,map,start_x,start_y)
        all_missed_squares.update(missed_squares)
    return all_missed_squares


In [178]:
def find_plan_dfs(map_data):
    # Directions: Up, Right, Down, Left
    dx = [0, 1, 0, -1]
    dy = [-1, 0, 1, 0]
    direction = ['N', 'E', 'S', 'W']

    height, width = len(map_data), len(map_data[0])
    #print("height width",height,width)
    start=find_start_position(map_data)
    #print("start", start)

    # Return if no start position is found
    if start[0]==None:
        possible_starts=find_potential_starts(map_data)
        # Choose a random start position if 'S' is not found
        start = random.choice(possible_starts)

    visited = set()
    plan = []

   # Flag to indicate completion
    all_cleaned = False

    def dfs(x, y):
        nonlocal all_cleaned
        if all_cleaned:  # If all squares are cleaned, return immediately
            return
        visited.add((x, y))
        for i in range(4):  # For each direction
            nx, ny = (x + dx[i]) % width , (y + dy[i]) % height
            #print(nx,ny)
            remaining_squares = check_all_clean(map_data, visited)
            if not remaining_squares:
                all_cleaned = True  # Set the flag when all squares are cleaned
                return
            #nx, ny = (x + dx) % width, (y + dy) % height
            if 0 <= nx < width and 0 <= ny < height and map_data[ny][nx] != 'X' and (nx, ny) not in visited:
                #print("appended ",direction[i])
                plan.append(direction[i])
                dfs(nx, ny)
                # Backtrack only if all squares are not cleaned
                if not all_cleaned:
                    #print("appended backtrack ",direction[(i+2)%4])
                    plan.append(direction[(i+2)%4])  # Append opposite direction

    dfs(*start)
    return ''.join(plan)

In [193]:
#Example
task, plan, map_data,start_x,start_y=read_map_from_file('problem_a_00.txt')
if task=="CHECK PLAN":
    if start_x==None:
        missed_squares = execute_plan_for_unknown_start(plan, map_data)
    else:
        missed_squares = execute_plan(plan, map_data,start_x,start_y)
    if not missed_squares:
        print("GOOD PLAN")
    else:
        print("BAD PLAN")
        print(missed_squares)
else:
    print(find_plan_dfs(map_data))
    

BAD PLAN
[(13, 6), (14, 6), (15, 6)]


In [207]:
import os

# Assuming the folder containing the problems is the current working directory.
# If not, replace with the correct path.
folder_path = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/el30atug-1/problems'
outputh_path = '/Users/melihekinci/Documents/FAU_Courses/ThirdSemester/el30atug-1/solutions'
suffixes = ['a', 'b', 'c', 'd', 'e', 'f']

def write_solution_to_file(filename, solution):
    with open(filename, 'w') as file:
        file.write(solution)

# Function to process each problem file
def process_file(suffix, index):
    filename = f"problem_{suffix}_{index:02}.txt" 
    output_filename = f"solution_{suffix}_{index:02}.txt"
    file_path = os.path.join(folder_path, filename)
    output_filepath = os.path.join(outputh_path, output_filename)

    if os.path.exists(file_path):
        task, plan, map_data, start_x, start_y = read_map_from_file(file_path)
        if task == "CHECK PLAN":
            if start_x == None:
                missed_squares = execute_plan_for_unknown_start(plan, map_data)
            else:
                missed_squares = execute_plan(plan, map_data, start_x, start_y)

            if not missed_squares:
                write_solution_to_file(output_filepath, "GOOD PLAN")
            else:
                with open(output_filepath, 'w') as file:
                    file.write("BAD PLAN\n")
                    for x in missed_squares:
                        file.write(str(x[0])+", "+str(x[1])+"\n")
        else:
            solution = find_plan_dfs(map_data)
            write_solution_to_file(output_filepath, solution)
        print(f"Processed {filename}")
    else:
        print(f"{filename} does not exist.")

# Loop through the files
suffixes = ['a', 'b', 'c', 'd', 'e', 'f']
for suffix in suffixes:
    for i in range(0, 20): 
        process_file(suffix, i)

Processed problem_a_00.txt
Processed problem_a_01.txt
Processed problem_a_02.txt
Processed problem_a_03.txt
Processed problem_a_04.txt
Processed problem_a_05.txt
Processed problem_a_06.txt
Processed problem_a_07.txt
Processed problem_a_08.txt
Processed problem_a_09.txt
Processed problem_a_10.txt
Processed problem_a_11.txt
Processed problem_a_12.txt
Processed problem_a_13.txt
Processed problem_a_14.txt
Processed problem_a_15.txt
Processed problem_a_16.txt
Processed problem_a_17.txt
Processed problem_a_18.txt
Processed problem_a_19.txt
Processed problem_b_00.txt
Processed problem_b_01.txt
Processed problem_b_02.txt
Processed problem_b_03.txt
Processed problem_b_04.txt
Processed problem_b_05.txt
Processed problem_b_06.txt
Processed problem_b_07.txt
Processed problem_b_08.txt
Processed problem_b_09.txt
Processed problem_b_10.txt
Processed problem_b_11.txt
Processed problem_b_12.txt
Processed problem_b_13.txt
Processed problem_b_14.txt
Processed problem_b_15.txt
Processed problem_b_16.txt
P